# Playing with Microsoft Defender for Endpoint (MDE) (old MDATP) REST API
https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/apis-intro

Requirements: the APP should have the following 'API permissions':
 - WindowsDefenderATP
   - ****.Read.All (for example AdvancedQuery.Read.All, Alert.Read.All, File.Read.All, ...)
   
**VERY** Important either the global admin allows these list of API permission OR you should give a link to him/her (NOTE to change the app clientid) :
$https://login.microsoftonline.com/common/oauth2/authorize?prompt=consent&client_id=<HERE-THE-CLIENT-ID>&response_type=code&sso_reload=true$

This REST API have 13 exposed APIs. We show bellow the link to each individual exposed API. Each API have multiple methods. 

|#|Topic|Description|Link|
|:-|:-|:-|:-|
|1|Advanced Hunting|Run queries from API.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/run-advanced-query-api)|
|2|Alerts|Run API calls such as get alerts, create alert, update alert and more.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/alerts)|
|3|Domains|Run API calls such as get domain-related devices, domain statistics and more.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/get-domain-related-alerts)|
|4|Files|Run API calls such as get file information, file related alerts, file related devices, and file statistics.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/files)|
|5|IPs|Run API calls such as get IP-related alerts and get IP statistics.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/get-ip-related-alerts)|
|6|Machines|Run API calls such as get devices, get devices by ID, information about logged on users, edit tags and more.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/machine)|
|7|Machine Actions|Run API call such as Isolation, Run anti-virus scan and more.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/machineaction)|
|8|Indicators|Run API call such as create Indicator, get Indicators and delete Indicators.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/ti-indicator)|
|9|Users|Run API calls such as get user-related alerts and user-related devices.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/user)|
|10|Score|Run API calls such as get exposure score or get device secure score.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/score)|
|11|Software|Run API calls such as list vulnerabilities by software.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/software)|
|12|Vulnerability|Run API calls such as list devices by vulnerability.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/vulnerability)|
|13|Recommendation|Run API calls such as Get recommendation by ID.|[link](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/recommendation)|

# Required information

In [1]:
appId='0fd88ba8-1455-4d87-8d73-6184b7601906'
tenantId='9daf1bf5-4bea-48be-bff4-9d8d19b77c68'
appSecret='ku_uuxs90v6bUM1Oor0~63N-OnG4VB_.2g'

# Required libraries

In [2]:
import pandas as pd
import json
import urllib.parse

<h1 align='center'>========================================================<br>=======================================================</h1>

# Function to get the token to access via the app

In [3]:
def mde_getaadtoken(tenantId, appId, appSecret):
    url = "https://login.windows.net/%s/oauth2/token" % (tenantId)
    resourceAppIdUri = 'https://api.securitycenter.windows.com'
    body = {
        'resource' : resourceAppIdUri,
        'client_id' : appId,
        'client_secret' : appSecret,
        'grant_type' : 'client_credentials'
    }
    data = urllib.parse.urlencode(body).encode("utf-8")
    req = urllib.request.Request(url, data)
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    aadToken = jsonResponse["access_token"]
    return aadToken

# Instantiating previous function

In [4]:
access_token = mde_getaadtoken(tenantId, appId, appSecret)

<h1 align='center'>========================================================<br>=======================================================<br> ADVANCED HUNTING API</h1>

Note: this API is the same as the of from the M365D. Examples of queries [here!](https://github.com/microsoft/Microsoft-365-Defender-Hunting-Queries)

# Function Run (KQL)
https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/run-advanced-query-api

In [5]:
def mde_advancedquery_run(access_token,kql_query):
    url = "https://api.securitycenter.microsoft.com/api/advancedqueries/run"
    req = urllib.request.Request(url, 
                                 headers={'Authorization' : "Bearer " + access_token, 'Content-Type':'application/json'},
                                 data=bytes(json.dumps({"Query": kql_query}), encoding="utf-8"))
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['Results'])

# Example of instantiating this function

In [6]:
kql_query = """
DeviceProcessEvents
|limit 10
"""

In [7]:
df_mde_advancedquery_run = mde_advancedquery_run(access_token,kql_query)
df_mde_advancedquery_run.head()

,Timestamp,DeviceId,DeviceName,ActionType,FileName,FolderPath,SHA1,SHA256,MD5,FileSize,...,InitiatingProcessCreationTime,InitiatingProcessFolderPath,InitiatingProcessParentId,InitiatingProcessParentFileName,InitiatingProcessParentCreationTime,InitiatingProcessSignerType,InitiatingProcessSignatureStatus,ReportId,AppGuardContainerId,AdditionalFields
0,2021-02-19T12:53:40.1887653Z,ef96648e47c6d74bb49de453d981359438fb6369,win10pro,ProcessCreated,SenseIR.exe,C:\Program Files\Windows Defender Advanced Thr...,b79faf73f4818f52012837ebad4e1f3381946b14,a33ba0fed96a74f81e698c4e27538a7d6d5da93cbcc4ca...,0af0fdcc87c2dae5191753024be45ad7,3760128,...,2021-02-15T15:01:16.7530145Z,c:\program files\windows defender advanced thr...,752,services.exe,2021-02-15T14:45:19.0273871Z,Unknown,Unknown,35092,,
1,2021-02-19T12:53:52.2239679Z,ef96648e47c6d74bb49de453d981359438fb6369,win10pro,ProcessCreated,powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0\pow...,f43d9bb316e30ae1a3494ac5b0624f6bea1bf054,9f914d42706fe215501044acd85a32d58aaef1419d404f...,04029e121a0cfa5991749937dd22a1d9,452608,...,2021-02-19T12:53:39.756309Z,c:\program files\windows defender advanced thr...,8668,MsSense.exe,2021-02-15T15:01:16.7530145Z,OsVendor,Valid,35095,,
2,2021-02-19T12:53:52.2242786Z,ef96648e47c6d74bb49de453d981359438fb6369,win10pro,ProcessCreated,conhost.exe,C:\Windows\System32\conhost.exe,9d48d4b78cbbfb22c9ee9070f713b35cd2a6a6eb,16c7a815a4a313d2c79816b3839376cc4d732dc0b136ee...,ce1a079265e7a92863baad92de538d72,843264,...,2021-02-19T12:53:51.6245894Z,c:\windows\system32\windowspowershell\v1.0\pow...,10604,SenseIR.exe,2021-02-19T12:53:39.756309Z,OsVendor,Valid,35096,,
3,2021-02-19T12:53:54.8443253Z,ef96648e47c6d74bb49de453d981359438fb6369,win10pro,ProcessCreated,csc.exe,C:\Windows\Microsoft.NET\Framework64\v4.0.3031...,528973416456c780051889ca1709510b6bf73370,4a6d0864e19c0368a47217c129b075dddf61a6a262388f...,f65b029562077b648a6a5f6a1aa76a66,2759232,...,2021-02-19T12:53:51.6245894Z,c:\windows\system32\windowspowershell\v1.0\pow...,10604,SenseIR.exe,2021-02-19T12:53:39.756309Z,OsVendor,Valid,35116,,
4,2021-02-19T12:53:54.8475361Z,ef96648e47c6d74bb49de453d981359438fb6369,win10pro,ProcessCreated,cvtres.exe,C:\Windows\Microsoft.NET\Framework64\v4.0.3031...,2a3249732f5aa4588a4a9895ffe217355041d663,1fe531eac592b480aa4bd16052b909c3431434f17e7ae1...,c877cbb966ea5939aa2a17b6a5160950,52744,...,2021-02-19T12:53:54.2442772Z,c:\windows\microsoft.net\framework64\v4.0.3031...,6608,powershell.exe,2021-02-19T12:53:51.6245894Z,OsVendorApplication,Valid,35117,,


<h1 align='center'>========================================================<br>=======================================================<br>ALERT API</h1>

List of methods in the Alert API (https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/alerts)

|Method	| Return Type |Description  |
|:- |:-|:-|
|Get alert|	Alert|	Get a single alert object.|
|List alerts|	Alert| collection	List alert collection.|
|Update alert|	Alert|	Update specific alert.|
|Create alert|	Alert|	Create an alert based on event data obtained from Advanced Hunting.|
|List related domains|	Domain collection|	List URLs associated with the alert.|
|List related files|	File collection|	List the file entities that are associated with the alert.|
|List related IPs|	IP collection|	List IPs that are associated with the alert.|
|Get related machines|	Machine|	The machine that is associated with the alert.|
|Get related users|	User|	The user that is associated with the alert.|

<h1 align='center'>...........................................................</h1>

# FUNCTION List alerts

In [ ]:
def mde_alerts_listalerts(access_token):
    url = "https://api.securitycenter.windows.com/api/alerts"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Instantiating the above function

In [ ]:
df_alerts = mde_alerts_listalerts(access_token)

# How the list of alerts looks like?

In [ ]:
df_alerts.head()

<h1 align='center'>...........................................................</h1>

# FUNCTION Get alert

In [ ]:
def mde_alerts_getalert(access_token,alert_id):
    url = "https://api.securitycenter.windows.com/api/alerts/"+str(alert_id)
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_dict(jsonResponse,orient='index').transpose()

# Instantiating the previous function

In [ ]:
mde_alerts_getalert(access_token,df_alerts['id'][0])

<h1 align='center'>...........................................................</h1>

# FUNCTION List related domains

In [ ]:
def mde_alerts_listrelateddomains(access_token,alarm_id):
    url = "https://api.securitycenter.windows.com/api/alerts/"+str(alarm_id)+"/domains"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Instantiating the previous function

In [ ]:
mde_alerts_listrelateddomains(access_token,df_alerts['id'][0])

<h1 align='center'>...........................................................</h1>


# FUNCTION List related files

In [ ]:
def mde_alerts_listrelatedfiles(access_token,alert_id):
    url = "https://api.securitycenter.windows.com/api/alerts/"+str(alert_id)+"/files"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Instantiating the previous function

In [ ]:
mde_alerts_listrelatedfiles(access_token,df_alerts['id'][0])

<h1 align='center'>...........................................................</h1>

# FUNCTION List related IPs

In [ ]:
def mde_alerts_listrelatedips(access_token,alarm_id):
    url = "https://api.securitycenter.windows.com/api/alerts/"+str(alarm_id)+"/ips"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Instantiating the previous function

In [ ]:
mde_alerts_listrelatedips(access_token,df_alerts['id'][0])

<h1 align='center'>...........................................................</h1>

# FUNCTION Get related machines

In [ ]:
def mde_alerts_getrelatedmachines(access_token,alarm_id):
    url = "https://api.securitycenter.windows.com/api/alerts/"+str(alarm_id)+"/machine"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_dict(jsonResponse,orient='index').transpose()

# Instantiating the previous function

In [ ]:
mde_alerts_getrelatedmachines(access_token,df_alerts['id'][0])

<h1 align='center'>========================================================<br>=======================================================</h1>

# List of methods in the Machine API
https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/machine

|Method|	Return Type|	Description|
|:- |:-|:-|
|List machines|	machine collection|	List set of machine entities in the org.|
|Get machine|	machine	|Get a machine by its identity.|
|Get logged on users|	user collection|	Get the set of User that logged on to the machine.|
|Get related alerts	|alert collection|	Get the set of alert entities that were raised on the machine.|
|Get installed software|	software collection|	Retrieves a collection of installed software related to a given machine ID.|
|Get discovered vulnerabilities|	vulnerability collection|	Retrieves a collection of discovered vulnerabilities related to a given machine ID.|
|Get security recommendations|	recommendation collection|	Retrieves a collection of security recommendations related to a given machine ID.|
|Add or Remove machine tags|	machine|	Add or Remove tag to a specific machine.|
|Find machines by IP|	machine collection|	Find machines seen with IP.|
|Find machines by tag|	machine collection|	Find machines by Tag.|
|Get missing KBs|	KB collection|	Get a list of missing KBs associated with the machine ID.|
|Set device value|	machine collection|	Set the value of a device.|

<h1 align='center'>...........................................................</h1>

# FUNCTION List machines

In [ ]:
def mde_machines_listmachines(access_token):
    url = "https://api.securitycenter.microsoft.com/api/machines"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Instantiating the previous function

In [ ]:
df_machines = mde_machines_listmachines(access_token)
df_machines

<h1 align='center'>...........................................................</h1>

# FUNCTION Get machines

In [ ]:
def mde_machines_getmachine(access_token,machine_id):
    url = "https://api.securitycenter.windows.com/api/machines/"+str(machine_id)
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_dict(jsonResponse,orient='index').transpose()

# Instantiating the previous function

In [ ]:
mde_machines_getmachine(access_token,df_machines['id'][0])